In [32]:
import pyodbc
from sqlalchemy import create_engine, text
import sqlalchemy.exc
from urllib import parse
import pandas as pd
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from datetime import datetime
from pretty_html_table import build_table



# Define Database Connection

CONNAS400 = """
Driver={iSeries Access ODBC Driver};
system=10.143.12.10;
Server=AS400;
Database=PROD;
UID=SMY;
PWD=SMY;
"""

CONNSQL = """
Driver={SQL Server};
Server=tn-sql;
Database=autodata;
UID=production;
PWD=Auto@matics;
"""

server = 'tn-sql'
database = 'autodata'
driver = 'ODBC+Driver+17+for+SQL+Server&AUTOCOMMIT=TRUE'
user = 'production'
pwd = parse.quote_plus("Auto@matics")
port = '1433'
database_conn = f'mssql+pyodbc://{user}:{pwd}@{server}:{port}/{database}?driver={driver}'
# Make Connection
engine = create_engine(database_conn)


In [39]:

def add_obs(df):
    """Add new Obs Spare Parts to the Obs Spare Parts Table"""
    table_name = 'tblObsSpares'
    df.to_sql(
        name=table_name,
        con=engine,
        schema='eng',
        if_exists='append',
        index=False,
        dtype= {
                "PartNum": sqlalchemy.types.VARCHAR(length=255),
             "EngPartNum": sqlalchemy.types.VARCHAR(length=255),
        }
    )


def get_inv():
    """Get Spare Inventory Data From iSeries AS400"""
    dbcnxn = pyodbc.connect(CONNAS400)
    cursor = dbcnxn.cursor()

    strsql = """SELECT PROD.FPSPRMAST1.SPH_PART,
                       STRIP(PROD.FPSPRMAST1.SPH_ENGPRT),
                       STRIP(PROD.FPSPRMAST1.SPH_DESC1),
                       STRIP(PROD.FPSPRMAST1.SPH_DESC2),
                       STRIP(PROD.FPSPRMAST1.SPH_MFG),
                       STRIP(PROD.FPSPRMAST1.SPH_MFGPRT),
                       STRIP(PROD.FPSPRMAST2.SPD_CABINT),
                       STRIP(PROD.FPSPRMAST2.SPD_DRAWER),
                       PROD.FPSPRMAST2.SPD_QOHCUR,
                       PROD.FPSPRMAST1.SPH_CURSTD,
                       STRIP(PROD.FPSPRMAST2.SPD_REODTE),
                       STRIP(PROD.FPSPRMAST2.SPD_USECC),
                       STRIP(PROD.FPSPRMAST2.SPD_PURCC),
                       STRIP(PROD.FPSPRMAST2.SPD_QREORD)
                FROM PROD.FPSPRMAST1 INNER JOIN PROD.FPSPRMAST2 ON PROD.FPSPRMAST1.SPH_PART = PROD.FPSPRMAST2.SPD_PART
                WHERE (((PROD.FPSPRMAST2.SPD_FACIL)=9))"""
    try:
        cursor.execute(strsql)
        result = cursor.fetchall()
    except Exception as e:
        msg = 'AS400 Inventory Query Failed: ' + str(e)
        result = []
        print(msg)
        print(strsql)
    else:
        msg = str(len(result)) + " AS400 Inventory Records Processed From Inventory Tables"
        print(msg)
    dbcnxn.close()
    return result

def find_new_obs_org(result_spares):
    """Find any newly obsoleted spare parts"""
    data_type_dict = {'StandardCost': float, 'OnHand': int, 'PartNum': str, 'ReOrderPt': int, 'ReOrderDate': int, 'Cabinet': str, 'Drawer': str}
    df_spares = pd.DataFrame.from_records(result_spares)
    df_spares.columns = ['PartNum', 'EngPartNum', 'Desc1', 'Desc2', 'Mfg', 'MfgPn', 'Cabinet', 'Drawer', 'OnHand',
                         'StandardCost', 'ReOrderDate', 'DeptUse', 'DeptPurch', 'ReOrderPt']
    df_spares = df_spares.dropna()
    df_spares = df_spares.astype(data_type_dict)
    df_spares = df_spares.convert_dtypes()
    df_obs_all = df_spares[df_spares.Cabinet.str.contains('OBS', case=False, na=False)]
    df_obs_current = pd.read_sql("SELECT PartNum FROM eng.tblObsSpares", engine)
    df_obs_new = df_obs_all[~df_obs_all['PartNum'].isin(df_obs_current['PartNum'])]
    return df_obs_new

def find_new_obs(result_spares):
    """Find any newly obsoleted spare parts"""
    data_type_dict = {'StandardCost': float, 'OnHand': int, 'PartNum': str, 'ReOrderPt': int,
                      'ReOrderDate': int, 'Cabinet': str, 'Drawer': str}
    # Ensure the correct structure of incoming data
    try:
        df_spares = pd.DataFrame.from_records(result_spares)
        expected_columns = ['PartNum', 'EngPartNum', 'Desc1', 'Desc2', 'Mfg', 'MfgPn',
                            'Cabinet', 'Drawer', 'OnHand', 'StandardCost', 'ReOrderDate',
                            'DeptUse', 'DeptPurch', 'ReOrderPt']
        if len(df_spares.columns) != len(expected_columns):
            raise ValueError("Mismatch in the number of columns in result_spares")
        df_spares.columns = expected_columns
    except Exception as e:
        print(f"Error creating DataFrame: {e}")
        return pd.DataFrame()  # Return an empty DataFrame in case of failure

    # Drop NaN values from critical columns
    df_spares = df_spares.dropna(subset=['PartNum', 'Cabinet', 'OnHand'])

    # Enforce data types safely
    for col, dtype in data_type_dict.items():
        if col in df_spares.columns:
            if dtype == int or dtype == float:
                df_spares[col] = pd.to_numeric(df_spares[col], errors='coerce')
            else:
                df_spares[col] = df_spares[col].astype(dtype)

    # Filter for obsolete parts
    df_obs_all = df_spares[df_spares.Cabinet.str.contains('OBS', case=False, na=False)]

    try:
        df_obs_current = pd.read_sql("SELECT PartNum FROM eng.tblObsSpares", engine)
    except Exception as e:
        print(f"Database query error: {e}")
        return pd.DataFrame()  # Return an empty DataFrame

    # Identify new obsolete parts
    df_obs_new = df_obs_all[~df_obs_all['PartNum'].isin(df_obs_current['PartNum'])]
    return df_obs_new


def send_email(to, subject, body, content_type='html', username='elab@idealtridon.com'):
    # Send Email
    mail_server = "cas2013.ideal.us.com"

    if isinstance(to, list):
        # Join the list of email addresses into a single string
        to = ', '.join(to)


    try:
    # Create a MIME email
        message = MIMEMultipart()
        message['From'] = username
        message['To'] = to
        message['Subject'] = subject
        start = """<html>
                <body>
                    <strong>Requested Spare Part(s):</strong><br />"""
        end = """       </body>
            </html>"""
        body = body + '<br><b>Sincerely,<br><br><br> The Engineering Overlords and Steve</b><br>'
        body = body + '<br><br><a href="https://www.idealtridon.com/idealtridongroup.html"> ' \
                        '<img src="https://sgilmo.com/email_logo.png" alt="Ideal Logo"></a>'
        # Attach the body content (HTML or plain text)
        message.attach(MIMEText(start+body+end, content_type))


        # Set up the SMTP connection
        with smtplib.SMTP(mail_server) as server:
            server.send_message(message)  # Send the email

        print(f"Email sent successfully to {to} with subject: {subject}")

    except Exception as e:
        print(f"Failed to send email: {e}")

def update_req():
    """Enter Timestamp for database records"""
    try:
        dbcnxn = pyodbc.connect(CONNSQL)
        cursor = dbcnxn.cursor()
        str_sql = """UPDATE dbo.tblReqSpare
                    SET dbo.tblReqSpare.reqdate = GETDATE()
                    WHERE dbo.tblReqSpare.reqdate IS NULL
                """

        # Execute the update
        cursor.execute(str_sql)
        dbcnxn.commit()
        print("Database updated successfully!")

        # Clean up resources
        cursor.close()
        dbcnxn.close()
    except pyodbc.Error as e:
        print(f"Database connection failed: {e}")




### Generate a Spare Parts Request

In [42]:

strsql = """
SELECT * FROM dbo.tblReqSpare
WHERE dbo.tblReqSpare.reqdate IS NULL
"""

mail_list = ['sgilmour@idealtridon.com']
width_list = ['300px', '2000px','1000px','400px','auto','auto','auto','auto','auto','auto','auto','auto'
              ,'auto','auto','auto','auto','auto']
# Ensure SQL is a string and trimmed
if not isinstance(strsql, str):
    raise TypeError("The SQL query must be a string.")
strsql = strsql.strip()


with engine.connect() as connection:
    df_reqspares = pd.read_sql_query(text(strsql), connection)
if not df_reqspares.empty:
    print("Dataframe Size = ",df_reqspares.size)
    df_reqspares['cost'] = df_reqspares['cost'].round(2)
    df_reqspares['reqdate'] = datetime.now().date()
    df_reqspares = df_reqspares[['req_by', 'depts_using', 'desc', 'mfg', 'vendor', 'mfgpn', 'dwg', 'rev', 'cost', 'qty_to_stock', 'qty_per_use', 'qty_annual_use', 'reorder_pt', 'reorder_amt' ]]
    # Renaming specific columns
    df_reqspares = df_reqspares.rename(columns={'req_by': 'Requested By', 'desc': 'Description',
                                                'mfgpn': 'Manu Part Number', 'dwg': 'Drawing',
                                                'rev': 'Revision','depts_using': 'Dept',
                                                'mfg': 'Manufacturer', 'vendor': 'Vendor',
                                                'cost': 'Cost', 'qty_to_stock': 'Stock',
                                                'qty_per_use': 'Used', 'qty_annual_use': 'Annual Usage',
                                                'reorder_pt': 'Reorder Pt', 'reorder_amt': 'Amount'})
    pretty_html = build_table(df_reqspares
                            , 'orange_dark'
                            , font_size='small'
                            , font_family='Arial'
                            , text_align='center'
                            , width= '100%'
                            , index=False)

# df_html_table = df_reqspares.to_html(index=False, classes='GenericTable')
    send_email(mail_list, 'Please Add The Following Spare Parts', pretty_html)
    update_req()
    print(df_reqspares.shape)


Dataframe Size =  54
Email sent successfully to sgilmour@idealtridon.com, steveg@sgilmo.com with subject: Please Add The Following Spare Parts
Database updated successfully!
(3, 14)
